In [ ]:
%load_ext autoreload
%autoreload 0

In [ ]:
%autoreload

In [ ]:
import sys
sys.path.append('/pollard/home/bsmith/Projects/haplo-benchmark/include/StrainFacts')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from lib.pandas_util import idxwhere
import sfacts as sf
import numpy as np
import xarray as xr
import seaborn as sns
from scipy.spatial.distance import cdist
import lib.plot
from tqdm import tqdm
from itertools import product

In [ ]:
# data/sfacts_simulate-model_ssdd-n1000-g10000-s20-rho10-pi100-mu10-eps10-alpha100-seed3.metagenotype-n100-g500.fit-sfacts36_cpu-s40-g10000-seed2.world
# stem = 'data/sfacts_simulate-model_ssdd-n1000-g10000-s20-rho10-pi10-mu10-eps10-alpha100-seed0'
# mgen_stem = 'metagenotype-n100-g100'
# sim0 = sf.World.load(f'{stem}.world.nc')
# mgen = sf.Metagenotypes.load(f'{stem}.{mgen_stem}.nc')

# sim = sim0.sel(position=mgen.position, sample=mgen.sample)
# fit_stem = 'fit-sfacts36_cpu-s40-g10000-seed0'
# fit_init = sf.World.load(f'{stem}.{mgen_stem}.{fit_stem}.world_initial.nc')
# fit = sf.World.load(f'{stem}.{mgen_stem}.{fit_stem}.world.nc')

benchmarks1 = []
for sim_seed, g, n, fit_type, fit_seed in tqdm(list(product(
    range(5),
    [500, 1000],
    [100, 200, 500, 1000],
    ['sfacts36_gpu-s40-g10000', 'sfacts36_cpu-s40-g10000', 'sfacts40_cpu-s40-g10000', 'sfinder-s20', 'sfinder-s30'],
    range(5),
))):
    sim_prefix = f"data/sfacts_simulate-model_ssdd-n1000-g10000-s20-rho10-pi10-mu100-eps10-alpha100-seed{sim_seed}"
    eval_path = f"{sim_prefix}.metagenotype-n{n}-g{g}.fit-{fit_type}-seed{fit_seed}.evaluation.tsv"
    try:
        bench = pd.read_table(eval_path, index_col='fit_path')
    except FileNotFoundError:
#         print(f"{fit_path} not found")
        continue
    
    meta_dict = dict(
        sim_seed=sim_seed,
        g=g,
        n=n,
        fit_type=fit_type,
        fit_seed=fit_seed,
        fit_s=fit_s,
    )
    for key in meta_dict:
        bench[key] = meta_dict[key]
    benchmarks1.append(bench)
benchmarks1 = pd.concat(benchmarks1)
print(benchmarks1.shape)

In [ ]:
# data/sfacts_simulate-model_ssdd-n1000-g10000-s20-rho10-pi10-mu100-eps10-alpha100-seed{0,1,2,3,4}.metagenotype-n{100,200,500}-g{500,1000}.fit-sfinder-s{20,30}-seed{0,1,2,3,4}.evaluation.tsv
# data/sfacts_simulate-model_ssdd-n1000-g10000-s20-rho10-pi10-mu100-eps10-alpha100-seed{0,1,2,3,4}.metagenotype-n{100,200,500}-g{500,1000}.fit-sfacts36_cpu-s40-g10000-seed{0,1,2,3,4}.evaluation.tsv
# data/sfacts_simulate-model_ssdd-n1000-g10000-s20-rho10-pi10-mu100-eps10-alpha100-seed{0,1,2,3,4}.metagenotype-n{500,1000}-g{500,1000,5000}.fit-sfacts36_gpu-s40-g10000-seed{0,1,2,3,4}.evaluation.tsv

In [ ]:
d0 = (
    benchmarks1
#     [lambda x: x.g == 500]
#     [lambda x: (
# #         (x.mu == 5)
# #         (x.fit_seed == 0)
#         (x.fit_type.isin([
#             'sfinder',
#             'sfacts1_cpu',
#             'sfacts1_gpu',
# #             'sfacts2',
# #             'sfacts3',
#         ]))
#     )]
    .groupby([
        'sim_seed',
        'g',
        'n',
        'fit_type',
        'fit_seed',
    ], as_index=False)
    .apply(lambda d: d.loc[d.metagenotype_prediction_error.idxmin()])
    .sort_values(['fit_type', 'g'])
)
# d0['fit_type_fit_s'] = d0['fit_type'] + '-s' + d0['fit_s'].astype(str)



palette = {
# 'sfinder', 'sfacts1_cpu', 'sfacts1_gpu', 'sfacts2', 'sfacts3'
    'sfacts36_cpu-s40-g10000': 'blue',
    'sfacts40_cpu-s40-g10000': 'lightblue',
    'sfacts36_gpu-s40-g10000': 'purple',
    'sfinder-s20': 'green',
    'sfinder-s30': 'lightgreen',
#     'sfacts1-s40': 'darkgreen',
#     'sfacts1_gpu-s20': 'lightgreen',  # 'lightsteelblue',
#     'sfacts1_gpu-s40': 'darkgreen',  # 'royalblue',
#     'sfacts2-s20': 'khaki',
#     'sfacts2-s40': 'darkgoldenrod',
#     'sfinder-s20': 'grey',
#     'sfinder-s40': 'black',
#     'sfacts7-s20': 'lightblue',
#     'sfacts7-s40': 'darkblue',
}

metric_list = [
#     ("metagenotype_prediction_error", dict(value='symlog', linthresh=1e-2, linscale=0.1), dict(bottom=-1e-3, top=1e0)),
    ("braycurtis_trans_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("unifrac_trans_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("unifrac_cis_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("rank_abundance_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("fwd_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("rev_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("fwd_discrete_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("rev_discrete_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("runtime", dict(value='log'), dict(bottom=1e1, top=1e6)),
          ]

g_list = [
#     50,
#     100,
#     250,
    500,
    1000,
]
n_list = [100, 200, 500, 1000]
ncol = len(g_list)
nrow = len(metric_list)
# nrow = int(np.ceil(len(metrics) / ncol))

fig, axs = plt.subplots(nrow, ncol, figsize=(5 * ncol, 2 * nrow), sharey='row', sharex=True)
axs = axs.reshape((nrow, ncol))

for (met, scale_kws, ylim_kws), row in zip(metric_list, axs):
    row[0].set_yscale(**scale_kws)
    row[0].set_ylim(**ylim_kws)
    row[0].set_ylabel(met)
    for g, ax in zip(g_list, row):
        d1 = d0[d0.g == g]
        ax.set_title(f'g={g}')
        sns.stripplot(
            x='n',
            y=met,
            data=d1,
            hue='fit_type',
            hue_order=[
                'sfacts36_gpu-s40-g10000',
                'sfacts36_cpu-s40-g10000',
                'sfacts40_cpu-s40-g10000',
                'sfinder-s20',
                'sfinder-s30',
            ],
            order=n_list,
            s=6,
            palette=palette,
            ax=ax,
            jitter=True,
            alpha=0.7,
            dodge=True,
        )
    
    
for ax in axs.flatten()[1:]:
    leg = ax.get_legend()
    if leg:
        leg.remove()
        
# for ax in axs[:-1].flatten():
#     ax.set_ylim(bottom=1e-4, top=1e0)
    
fig.tight_layout()

In [ ]:
# data/sfacts_simulate-model_simplest_simulation-n{n}-g{{g}}-s{{s}}-{{sim_params}}.metagenotype-n{n}-g{{g}}.fit-{{fit_type}}-s{fit_s}-seed{{seed}}.evaluation.tsv
g = 250
n_to_s_ratio = 5

_benchmarks = []
for s, fit_type, sim_seed, fit_seed, fit_s_ratio in tqdm(list(product(
    [10, 20, 40, 80, 200, 500],
    ['sfinder', 'sfacts40_gpu', 'sfacts40_cpu', 'sfacts40_big'],
    range(5),
    range(5),
    [1, 2],
))):
    n = s * n_to_s_ratio
    sim_prefix = f"data/sfacts_simulate-model_simplest_simulation-n{n}-g{g}-s{s}-pi40-mu100-eps10-seed{sim_seed}"
    mgen_prefix = f"{sim_prefix}.metagenotype-n{n}-g{g}"
    fit_s = s * fit_s_ratio
    eval_path = f"{mgen_prefix}.fit-{fit_type}-s{fit_s}-seed{fit_seed}.evaluation.tsv"
    try:
        bench = pd.read_table(eval_path, index_col='fit_path')
    except FileNotFoundError:
#         print(f"{fit_path} not found")
        continue
    
    meta_dict = dict(
        sim_seed=sim_seed,
        fit_seed=fit_seed,
        n=n,
        s=s,
        fit_s_ratio=fit_s_ratio,
        fit_s=fit_s,
        fit_type=fit_type,
    )
    for key in meta_dict:
        bench[key] = meta_dict[key]
    _benchmarks.append(bench)
_benchmarks = pd.concat(_benchmarks)
print(_benchmarks.shape)
benchmarks2 = _benchmarks

In [ ]:
d0 = (
    benchmarks2
#     [lambda x: x.g == 500]
#     [lambda x: (
# #         (x.mu == 5)
# #         (x.fit_seed == 0)
#         (x.fit_type.isin([
#             'sfinder',
#             'sfacts1_cpu',
#             'sfacts1_gpu',
# #             'sfacts2',
# #             'sfacts3',
#         ]))
#     )]
    .groupby([
        's',
        'fit_type',
        'sim_seed',
        'fit_s_ratio',
#         'fit_seed',
    ], as_index=False)
    .apply(lambda d: d.loc[d.metagenotype_prediction_error.idxmin()])
)
# d0['fit_type_fit_s'] = d0['fit_type'] + '-s' + d0['fit_s'].astype(str)



palette = {
# 'sfinder', 'sfacts1_cpu', 'sfacts1_gpu', 'sfacts2', 'sfacts3'
    'sfacts40_cpu': 'purple',
    'sfacts40_gpu': 'blue',
    'sfacts40_big': 'lightblue',
    'sfinder': 'green',
}

metric_list = [
#     ("metagenotype_prediction_error", dict(value='symlog', linthresh=1e-2, linscale=0.1), dict(bottom=-1e-3, top=1e0)),
    ("braycurtis_trans_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("unifrac_trans_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("unifrac_cis_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("rank_abundance_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("fwd_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("rev_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("fwd_discrete_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("rev_discrete_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("runtime", dict(value='log'), dict(bottom=1e1, top=1e6)),
          ]

g_list = [
#     50,
#     100,
#     250,
    500,
    1000,
]
s_list = [10, 20, 40, 80, 200, 500]
fit_s_ratio_list = [1, 2]
ncol = len(fit_s_ratio_list)
nrow = len(metric_list)
# nrow = int(np.ceil(len(metrics) / ncol))

fig, axs = plt.subplots(nrow, ncol, figsize=(5 * ncol, 2 * nrow), sharey='row', sharex=True)
axs = axs.reshape((nrow, ncol))

for (met, scale_kws, ylim_kws), row in zip(metric_list, axs):
    row[0].set_yscale(**scale_kws)
    row[0].set_ylim(**ylim_kws)
    row[0].set_ylabel(met)
    for fit_s_ratio, ax in zip(fit_s_ratio_list, row):
        d1 = d0[d0.fit_s_ratio == fit_s_ratio]
        ax.set_title(f'fit_s_ratio={fit_s_ratio}')
        sns.stripplot(
            x='s',
            y=met,
            data=d1,
            hue='fit_type',
            hue_order=[
                'sfacts40_cpu',
                'sfacts40_gpu',
                'sfacts40_big',
                'sfinder',
            ],
            order=s_list,
            s=6,
            palette=palette,
            ax=ax,
            jitter=True,
            alpha=0.7,
            dodge=True,
        )
    
    
for ax in axs.flatten()[1:]:
    leg = ax.get_legend()
    if leg:
        leg.remove()
        
# for ax in axs[:-1].flatten():
#     ax.set_ylim(bottom=1e-4, top=1e0)
    
fig.tight_layout()

In [ ]:
# Aggregate simulation results by finding the mean/min/std of each fit_seed
# and then taking the mean of each of these values over the sim_seed.
# I could then plot each runtime value as the mean


palette = {
# 'sfinder', 'sfacts1_cpu', 'sfacts1_gpu', 'sfacts2', 'sfacts3'
    'sfacts40_cpu': 'blue',
    'sfacts40_gpu': 'purple',
    'sfacts40_big': 'lightblue',
    'sfinder': 'green',
}

d0 = (
    benchmarks2
    .groupby([
        'n',
        'fit_type',
        'sim_seed',
        'fit_s_ratio',
    ])
    .agg(['mean', 'min', 'std', 'count'])
    .reset_index()
    .groupby([
        'n',
        'fit_type',
        'fit_s_ratio',
    ])
    .agg(['mean', 'count'])
    ['runtime']
    .reset_index()
)

fig = plt.figure(figsize=(7, 4))
for fit_type, d1 in d0[lambda x: x.fit_s_ratio==1].sort_values('n').groupby('fit_type'):
    if fit_type == 'sfacts40_big':
        continue
    plt.plot(d1['n'], d1[('min', 'mean')], c=palette[fit_type], label=fit_type)
for fit_type, d1 in d0[lambda x: x.fit_s_ratio==2].sort_values('n').groupby('fit_type'):
    if fit_type == 'sfacts40_big':
        continue
    plt.plot(d1['n'], d1[('min', 'mean')], c=palette[fit_type], linestyle='--')
    
plt.plot([], [], c='grey', linestyle='-', label='strains = samples * 0.2')
plt.plot([], [], c='grey', linestyle='--', label='strains = samples * 0.4')

plt.yscale('log')
plt.xscale('log')
plt.legend()

plt.ylabel('runtime (sec)')
plt.xlabel('samples')

In [ ]:
d0

In [ ]:
# Aggregate simulation results by finding the mean/min/std of each fit_seed
# and then taking the mean of each of these values over the sim_seed.
# I could then plot each runtime value as the mean


palette = {
# 'sfinder', 'sfacts1_cpu', 'sfacts1_gpu', 'sfacts2', 'sfacts3'
    'sfacts40_cpu': 'blue',
    'sfacts40_gpu': 'purple',
    'sfacts40_big': 'lightblue',
    'sfinder': 'green',
}

d0 = (
    benchmarks2
    .groupby([
        's',
        'fit_type',
        'sim_seed',
        'fit_s_ratio',
    ])
    .agg(['mean', 'min', 'std', 'count'])
    .reset_index()
    .groupby([
        's',
        'fit_type',
        'fit_s_ratio',
    ])
    .agg(['mean', 'count'])
)

d1 = d0.reset_index()[lambda x: (x.s==40) & (x.fit_type.isin(['sfacts40_cpu', 'sfinder']))]
metric_list = [
#     ("metagenotype_prediction_error", dict(value='symlog', linthresh=1e-2, linscale=0.1), dict(bottom=-1e-3, top=1e0)),
    ("braycurtis_trans_error", dict(value='symlog', linthresh=1e-4, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("unifrac_trans_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("unifrac_cis_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("rank_abundance_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("fwd_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("rev_genotype_error", dict(value='symlog', linthresh=1e-3, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("fwd_discrete_genotype_error", dict(value='symlog', linthresh=1e-4, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
    ("rev_discrete_genotype_error", dict(value='symlog', linthresh=1e-4, linscale=0.1), dict(bottom=-1e-4, top=1e0)),
#     ("runtime", dict(value='log'), dict(bottom=1e1, top=1e6)),
          ]

fig, axs = plt.subplots(1, len(metric_list), figsize=(2 * len(metric_list), 4))

for (met, scale_kws, ylim_kws), ax in zip(metric_list, axs):
    ax.set_ylabel(met)
    ax.set_yscale(**scale_kws)
    ax.set_ylim(**ylim_kws)
    ax.set_xticks([1, 2])
    ax.set_xticklabels(['exact', '2x'])
#     ax.set_ylim(1e-5)
    for fit_s_ratio, d2 in d1.groupby(['fit_s_ratio']):
        for fit_type, offset in zip(['sfacts40_cpu', 'sfinder'], [-0.1, 0.1]):
            d3 = d2[lambda x: (x.fit_type == fit_type)]
            if d3.empty:
                continue
            ax.scatter([fit_s_ratio + offset], d3[(met, 'min', 'mean')], edgecolor=palette[fit_type], color='white')
            ax.scatter([fit_s_ratio + offset], d3[(met, 'mean', 'mean')], color=palette[fit_type])
#             ax.vlines([fit_s_ratio + offset], d3[(met, 'min', 'mean')], d3[(met, 'mean', 'mean')], color=palette[fit_type])
            ax.vlines([fit_s_ratio + offset], d3[(met, 'mean', 'mean')] - d3[(met, 'std', 'mean')], d3[(met, 'mean', 'mean')] + d3[(met, 'std', 'mean')], color=palette[fit_type])
    
    
fig.tight_layout()

In [ ]:
plt.plot([40, 200, 1000], [432, 476, 716], label=250, lw=1.5, color='blue')
plt.plot([40, 200, 1000], [492, 674, 1740], label=2500, lw=3, color='darkblue')
plt.legend(title='g')
plt.yscale('log')
# plt.xscale('log')
plt.ylabel('Maximum GPU Memory Allocated (Mb)')
plt.xlabel('nstrains (nsamples=strains * 2.5)')

In [ ]:
# data/sfacts_simulate-model_simplest_simulation-n10000-g1000-s5-pi40-mu100-eps10-seed0.metagenotype-n{100,500,1000,10000}-g{250,500,1000}.fit-sfacts40_gpumem-s{20,40,100,200,400}-seed0.gpumem

_benchmarks = []
for n, s, g in tqdm(list(product(
    [100, 500, 1000, 10000],
    [20, 40, 100, 200, 400],
    [250, 500, 1000],
))):
    sim_prefix = f"data/sfacts_simulate-model_simplest_simulation-n10000-g1000-s5-pi40-mu100-eps10-seed0"
    mgen_prefix = f"{sim_prefix}.metagenotype-n{n}-g{g}"
    eval_path = f"{mgen_prefix}.fit-sfacts40_gpumem-s{s}-seed0.gpumem"
#     print(eval_path)
    try:
        with open(eval_path) as f:
            max_vmem = max([int(line) for i, line in enumerate(f)])
    except FileNotFoundError:
        print(f"Not found: {eval_path}")
    else:
        meta_dict = dict(
            n=n, s=s, g=g, max_vmem=max_vmem,
        )
        _benchmarks.append(meta_dict)
_benchmarks = pd.DataFrame(_benchmarks)
print(_benchmarks.shape)
benchmarks3 = _benchmarks

In [ ]:
d0 = benchmarks3.assign(n_to_s_ratio=lambda x: x.n / x.s)
ordered_s_list = [20, 40, 100, 200, 400]
palette = {s: v for s, v in zip(ordered_s_list, mpl.cm.cool(ordered_s_list))}
ls_map = {250: ':', 500: '--', 1000: '-'}

for (s, g), d1 in d0.sort_values('n').groupby(['s', 'g']):
    plt.plot('n', 'max_vmem', ls=ls_map[g], c=palette[s], data=d1, label='__nolegend__')
    
for s in ordered_s_list:
    plt.plot([], [], ls='-', c=palette[s], label=f's={s}')
for g in d0.g.unique():
    plt.plot([], [], ls=ls_map[g], c='black', label=f'g={g}')
plt.legend(loc='upper left')

plt.yscale('log')
plt.xscale('log')
plt.ylabel('Peak GPU Memory Allocation (Mb)')
plt.xlabel('samples')
# plt.ylim(4e2, 5e3)

In [ ]:
d0 = benchmarks3.assign(n_to_s_ratio=lambda x: x.n / x.s)
palette = {
    'sfacts40': 'blue',
    'sfinder': 'green',
}

fig, axs = plt.subplots(1, 2, sharex=True, sharey=True)
for (g, d1), ax in zip(d0.groupby('g'), axs):
    ax.set_title(f'g={g}')
    for n_to_s_ratio, ls in zip([5, 2.5], ['-', '--']):
        d2 = d1[d1.n_to_s_ratio == n_to_s_ratio]
        ax.plot('n', 'max_vmem', ls=ls, c=palette['sfacts40'], data=d2, label='__nolegend__')
        
        
ax.set_yscale('log')
ax.set_xscale('log')

for n_to_s_ratio, ls, ratio in zip([5, 2.5], ['-', '--'], [0.2, 0.4]):
    ax.plot([], [], c='black', ls=ls, label=f'strains = samples * {ratio}')
plt.legend()
    
    

# for (s, g), d1 in d0.sort_values('n').groupby(['s', 'g']):
#     plt.plot('n', 'max_vmem', ls=ls_map[g], c=palette[s], data=d1, label='__nolegend__')
    
# for s in ordered_s_list:
#     plt.plot([], [], ls='-', c=palette[s], label=f's={s}')
# for g in d0.g.unique():
#     plt.plot([], [], ls=ls_map[g], c='black', label=f'g={g}')
# plt.legend(loc='upper left')

# plt.yscale('log')
# plt.xscale('log')
# plt.ylabel('Peak GPU Memory Allocation (Mb)')
# plt.xlabel('samples')
# # plt.ylim(4e2, 5e3)

In [ ]:
# data/sfacts_simulate-model_simplest_simulation-n1000-g1000-s5-pi40-mu100-eps10-seed0.metagenotype-n{100,200,500}-g{250,500}.fit-{sfacts40,sfinder}_timeit-s{20,40,100,200}-seed0.time

import re

_benchmarks = []
for n, g, fit_type, s in tqdm(list(product(
    [100, 200, 500],
    [250, 500],
    ['sfacts40', 'sfinder'],
    [20, 40, 100, 200],
))):
    sim_prefix = f"data/sfacts_simulate-model_simplest_simulation-n1000-g1000-s5-pi40-mu100-eps10-seed0"
    mgen_prefix = f"{sim_prefix}.metagenotype-n{n}-g{g}"
    eval_path = f"{mgen_prefix}.fit-{fit_type}_timeit-s{s}-seed0.time"
#     print(eval_path)
    try:
        with open(eval_path) as f:
            d = f.readlines()
    except FileNotFoundError:
        print(eval_path)
        continue
    else:
        if len(d) == 0:
            print(eval_path)
            continue
        maxrss = int(re.match('.* ([0-9]+)maxresident\)k', d[0])[1])
    meta_dict = dict(
        n=n,
        g=g,
        s=s,
        fit_type=fit_type,
        maxrss=maxrss,
    )
    _benchmarks.append(meta_dict)
_benchmarks = pd.DataFrame(_benchmarks)
print(_benchmarks.shape)
benchmarks4 = _benchmarks

In [ ]:
d0 = benchmarks4.assign(n_to_s_ratio=lambda x: x.n / x.s, maxrss_mb=lambda x: x.maxrss / 1e3)
ordered_s_list = [20, 40, 100, 200, 400]

ls_map = {250: ':', 500: '--', 1000: '-'}

fig, ax = plt.subplots()
for fit_type, cm in zip(['sfinder', 'sfacts40'], [mpl.cm.summer, mpl.cm.cool]):
    palette = {s: v for s, v in zip(ordered_s_list, cm(ordered_s_list))}
    for (s, g), d1 in d0[d0.fit_type == fit_type].sort_values('n').groupby(['s', 'g']):
        ax.plot('n', 'maxrss_mb', ls=ls_map[g], c=palette[s], data=d1, lw=lw, label='__nolegend__')
    for s in d0.s.sort_values().unique():
        ax.plot([], [], ls='-', c=palette[s], label=f'{fit_type} (s={s})')

ax.legend(ncol=4, bbox_to_anchor=(1.0, 1.0))

ax.set_yscale('log')
ax.set_xscale('log')
ax.set_ylabel('Peak Main Memory Allocation (Mb)')
ax.set_xlabel('samples')
ax.set_ylim(5e1, 1e3)
fig.tight_layout()